In [ ]:
%matplotlib nbagg
import matplotlib.pyplot as plt
from IPython.display import display
##
import json
import re
import numpy           as np
import pandas          as pd
import statsmodels.api as sm
import ipywidgets      as widgets
##
import thundermint.logs  as Log
import thundermint.plot  as plot
import thundermint.splot as splot

In [ ]:
logs = Log.load_logs_files('../thundermint/logs/')
p = plot.CommitData(logs)

In [ ]:
class Tabs(object):
    def __init__(self, names):
        self.names = names
    def __enter__(self):
        outs = [widgets.Output() for _ in self.names]
        tab  = widgets.Tab(children=outs)
        for i,nm in enumerate(self.names):
            tab.set_title(i,nm)
        display(tab)
        return outs
    def __exit__(self, type, value, trace):
        pass

# Plots

In [ ]:
with Tabs(["Height", "Residuals", "Round"]) as tabs:
    with tabs[0]:
        with plot.LegendPlot() as ax:
            p.plot_points(ax, reltime=True)
    with tabs[1]:
        with plot.LegendPlot() as ax:
            p.plot_residuals_TvsH(ax)
    with tabs[2]:
        plot.plot_round(logs)

In [ ]:
with Tabs(["Block size", "Block size (hist)", "N sigs"]) as tabs:
    with tabs[0]:
        with plot.SimplePlot() as ax:
            p.plot_ntx(ax)
    with tabs[1]:
        with plot.SimplePlot() as ax:
            p.plot_ntx_distr(ax)
    with tabs[2]:
        with plot.SimplePlot() as ax:
            p.plot_n_signatures(ax)

In [ ]:
with Tabs(['mempool size', 'filtering time']) as tabs:
    with tabs[0]:
        plot.plot_mempool_size(logs)
    with tabs[1]:
        plt.figure()
        plt.grid()
        plt.title("Filtering time")
        df   = logs['node-1.log'].mempool
        dfA  = df[df['msg'] == 'Mempool after filtering']
        dfB  = df[df['msg'] == 'Mempool before filtering']
        fltT = (dfA['at'].values - dfB['at'].values).astype('timedelta64[ms]').astype(float)
        plt.hist(fltT, bins=50)

In [ ]:
splot.splot(logs, w=8096)

# Playground

In [ ]:
# Log keys
keyNH   = "Entering new height ----------------"
keyPrp  = "Entering propose"
keyPV   = "Entering prevote"
keyPC   = "Entering precommit"
keyCmt  = "Decision to commit"
keyDone = "Actual commit"
keyOrdSet = [keyNH, keyPrp, keyPV, keyPC, keyCmt, keyDone]
keySet = frozenset(keyOrdSet)

In [ ]:
df       = logs['node-1.log'].cons.copy()
df['H']  = df['data'].apply(lambda x: x.get('H'))
df       = df[df['H']>0]
df       = df[df['msg'].isin(keySet)]
deltaT   = (df['at'].values[1:] - df['at'].values[:-1]).astype('timedelta64[ms]')
df       = df[:-1]
df['dt'] = deltaT.copy()
steps    = {k:v for k,v in df.groupby(['msg'])}

In [ ]:
for k in keyOrdSet:
    v = steps[k]
    plt.figure()
    plt.grid()
    #plt.semilogy()
    plt.title(k)
    plt.hist(v['dt'].values.astype('float')/1e6, bins=100)

In [ ]:
plt.figure()
plt.grid()
plt.semilogy()
ys,xs,_ = plt.hist(steps['Entering new height ----------------']['dt'].values.astype('float')/1e6, bins=100)
ys = np.log(ys)
xs = (xs + (xs[1] - xs[0])/2)[:-1]
xs = xs[ys>0]
ys = ys[ys>0]
rr = sm.OLS(ys, sm.add_constant(xs), missing="drop").fit()
plt.plot(xs, np.exp((xs*rr.params[1] + rr.params[0])))
plt.title('Entering new height. N~exp(-t/%.3g)' % (-1/rr.params[1]))
None